In [ ]:
!pip install ultralytics

In [ ]:
#requirements
import glob
import shutil
import yaml
import os
import re
from tqdm import tqdm
import random
import ultralytics
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/cj_data'

In [ ]:
#Unzip Files
# %cd /content/drive/MyDrive/cj_data
# #!unzip "Data.zip"
# !unzip "labels.zip"
# %cd /content

In [ ]:
image_len = (glob.glob(f'{path}/data/image/*.jpg'))
len(image_len)

596

In [ ]:
labels_len = (glob.glob(f'{path}/labels/*.txt'))
len(labels_len)

589

In [ ]:
validation_len = (glob.glob(f'{path}/data/validation/**/*.jpg'))
len(validation_len)


45

### making Yaml File

In [ ]:
with open(f'{path}/data/labeling.txt', 'r', encoding='utf-8') as la:
    next(la)
    lines = la.readlines()
    classes = [(line.strip().split('\t')) for line in lines]


In [ ]:
labels_lst = []
labels = []
for cl in classes:
    labels_lst.append((str(cl[0])+ '_' + cl[1]))
    labels.append(cl[0])

In [ ]:
data_yaml = {
    'nc':len(classes),
    'names': labels,
    'train': '/content/drive/MyDrive/cj_data/data/train',
    'val': '/content/drive/MyDrive/cj_data/data/val',
    'test': '/content/drive/MyDrive/cj_data/data/test'
}
with open(f'{path}/data/data.yaml', 'w', encoding='utf-8') as f:
    yaml.dump(data_yaml, f,allow_unicode=True)

In [ ]:
!cat /content/drive/MyDrive/cj_data/Data/data.yaml

### make datasets

In [ ]:
# cj_data
# ㄴ Data
#     ㄴimage# 기존 이미지 데이터 모아놓은 폴더
#     ㄴimg   #labels별로 이미지 분리하여 폴더에 저장
#     ㄴtrain #train 이미지를 저장
#     ㄴval   #val 이미지를 저장
#     ㄴtest

In [ ]:
labels = []
for cl in data_yaml['names']:
    labels.append(cl)

In [ ]:
#초기화 할 경우
shutil.rmtree('/content/drive/MyDrive/cj_data/data/img')
shutil.rmtree('/content/drive/MyDrive/cj_data/data/train')
shutil.rmtree('/content/drive/MyDrive/cj_data/data/val')
# shutil.rmtree('/content/drive/MyDrive/cj_data/data/test')

#making Datasets dir
if not os.path.exists('/content/drive/MyDrive/cj_data/data/img'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/img')

if not os.path.exists('/content/drive/MyDrive/cj_data/data/train'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/train')

if not os.path.exists('/content/drive/MyDrive/cj_data/data/val'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/val')

if not os.path.exists('/content/drive/MyDrive/cj_data/data/test'):
    os.makedirs('/content/drive/MyDrive/cj_data/data/test')

#labels dir

for dir in labels:
    os.makedirs(os.path.join('/content/drive/MyDrive/cj_data/data/img', dir), exist_ok=True)

for dir in labels:
    os.makedirs(os.path.join('/content/drive/MyDrive/cj_data/data/train', dir), exist_ok=True)

for dir in labels:
    os.makedirs(os.path.join('/content/drive/MyDrive/cj_data/data/val', dir), exist_ok=True)

In [ ]:
#img 폴더로 분류
image_data = sorted(glob.glob(f'{path}/data/image/*.jpg'))
image_data.sort()
for train_image in tqdm(image_data, desc = 'Copying to img folder'):
    num_start = re.search(r'\d+', train_image)
    num = train_image[num_start.span()[0]:-6]
    shutil.copy(train_image, f'/content/drive/MyDrive/cj_data/data/img/{num}')


In [ ]:
data_folder = '/content/drive/MyDrive/cj_data/data/img'
train_folder = '/content/drive/MyDrive/cj_data/data/train'
val_folder = '/content/drive/MyDrive/cj_data/data/val'

folder_list = os.listdir(data_folder)

for folder in folder_list:
    folder_path = os.path.join(data_folder, folder)
    files = os.listdir(folder_path)

    if len(files) >= 2:
        val_file = random.choice(files)
        val_file_path = os.path.join(folder_path, val_file)
        new_val_file_path = os.path.join(val_folder, folder, val_file)
        shutil.move(val_file_path, new_val_file_path)

for folder in folder_list:
    folder_path = os.path.join(data_folder, folder)
    files = os.listdir(folder_path)

    for file in files:
        file_path = os.path.join(folder_path, file)
        new_file_path = os.path.join(train_folder, folder, file)
        shutil.move(file_path, new_file_path)

In [ ]:
#Check files
train_len = glob.glob(f'{path}/data/train/**/*.jpg')
val_len = glob.glob(f'{path}/data/val/**/*.jpg')
print(len(train_len), len(val_len))

In [ ]:
model = ultralytics.YOLO('yolov8m-cls')
result = model.train(
    data = '/content/drive/MyDrive/cj_data/Data/',
    imgsz = 640,
    epochs = 25,
    batch=32,
    patience=25,
    warmup_epochs=0,
    lr0=0.01,
    device=0,
    project='/content/drive/MyDrive/cj_data/yolo/results',
    name='exp',
    seed=74,
    optimizer='SGD',#Adam, AdamW, Adamax
    val=True,
    cache=False,
    pretrained=True,
    save_period=5,
    resume=False,
    degrees=45,
    translate=0.5,
    scale=0.3,
    mixup=0.2

)

In [ ]:
BATCH_SIZE=45
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]


In [ ]:
#get validation img
valid = glob.glob(f'/content/drive/MyDrive/cj_data/Data/validation/**/*.jpg')
print(len(valid))
valid

In [ ]:

#Predict
model = ultralytics.YOLO('yolov8m-cls')
model = ultralytics.YOLO('/content/drive/MyDrive/cj_data/yolo/results/exp14/weights/best.pt')
for i, image in tqdm(enumerate(get_test_image_paths(valid)), total=int(len(valid)/BATCH_SIZE)):
    print(i, len(image))
    model.predict(source=image, imgsz=640, iou=0.5, conf=0.2, save=True, save_conf=True, save_txt=True, project='/content/drive/MyDrive/cj_data/yolo/pred', name='exp', exist_ok=False, device=0, verbose=True, show_conf=False)
